# Implementation of the steps of the round 2

#### Let's write the steps 7b and 8b of the paper

**Step 7b**
* Add one ancilla $b$
* Use CNOTS to compute
$$
|r \cdot x_0 \rangle_b |x_0\rangle_x + |r \cdot x_1 \rangle_b | x_1\rangle_x
\tag{1}
$$
where $r\cdot x$ is bitwise inner product.

**Step 8b**
* Measure $x$ register in Hadamard basis yelding a string $d$.
* Discard $x$, the state is now
$$
|\psi \rangle_b \in \{|0\rangle, | 1\rangle,|+\rangle,|-\rangle\}
\tag{2}
$$

#### A couple of notes about the "bitwise inner product"
* Often and interchangebly, the bitwise inner prodcut is also called "binary" inner product, or also bitwise (or binary) "dot" product.
* The paper does not mention it, but since the ancilla register is constituted by only one qubit, the bitwise inner product has to be modulo 2. Indeed, that bitwise inner product should be written as $(r \cdot x) \mod 2$ (as written in some Classiq documentation).
  Thus the sum in the bitwise inner product, over the bit digits, should be modulo 2. Or, equivalently, it should be the result of a sequence of $\oplus$ (addition modulo 2, or XOR) operations (i.e. the sum of the bit products, $p_0$, $p_1$, $p_2$, $\dots$ should be $((p_0 \oplus p_1) \oplus p_2) \oplus$ $\dots$ and so on).

#### Let's examine how to implement the step 7b

**Before the step 7b**, the state of the circuit is given by the superposition in the $x$ register
$$ (|x_0\rangle + | x_1\rangle)_x ,
\tag{3}
$$
(we have already discarded the $y$ register).

We add an ancilla register $b$ constituted by only one qubit, so the state of the circuit is
$$ | 0 \rangle (|x_0\rangle + | x_1\rangle)_x .
\tag{4}
$$

So we need to compute the bitwise inner product between $x$ and $r$ and store it in the ancilla qubit.

Note that since state of the $x$ register is in superosition between the states $|x_0\rangle$ and $| x_1\rangle$, if the unitary operation of computation of the prodcut is controlled by the state of the register $x$, automatically we obtain the desired state (Eq. 1).

The bitwise inner product $(r\cdot x) \ \textrm{mod} \, 2$ is the defined as the sum of the product beteween respective bits of $r$ and $x$:
$$
(r \cdot x) \ \textrm{mod} \, 2 = \sum_{i=0}^{M} r_i x_i \ \textrm{mod} \, 2
$$
where $M$ is the size of the bit strings $r$ and $x$ (by protocol contruction $r$ is chosen to be the same size of $x$) and $r_i,x_i \in \{0,1\}$ are the bit values in the strings.

Each bit product $r_i x_i $ is $1$ only if both $r_i$ and $x_i$ are $1$ ($r_i =x_i =1$), and $0$ in the other three cases. So to implement each bit product $r_i x_i $ we can use a simple CNOT controlled by $x_i$ when $r_i = 1$. So the CNOTs are applied in the circuit only to the qubits $\{i\}$ for wich $r_i$ is equal to $1$.

The modulo 2 sum is automatic, since every CNOT negates the current state of ancilla qubit. So the first CNOT produces 1, the second negates 1, so produces 0, i.e. has performed the sum $1+1$ modulo 2, and so on.

#### Implemenation of the step 6b (for Cirq)

In [1]:
from random import randrange

Let's choose randomly a bit string (a list of 0 and 1) for $r$.

In [2]:
m=5                            # it is just an example
r = [randrange(0,2) for i in range(m)]
print(r)

[1, 1, 1, 0, 0]


#### Implemenation of the step 7b using Cirq

It is a basic implementation, just to illustrate the circuit logic

In [ ]:
# note: in this moment the code does not work, however now it is not relevant
import cirq
circuit = cirq.Circuit()
q_x = cirq.LineQubit.range(m) # qubits of x register
q_b = cirq.LineQubit.range(1) # ancilla qubit b 
# I append the CNOT to the circuit conditioned by the value of r_i (r_i = 1)
for i in range(len(r)):
    if r[i] == 1:
        circuit.append(cirq.CNOT(q_x[i], q_b))

#### Implemenation of the step 7b using Classiq Python SDK

For the implementation to Classiq SDK we can exploit the high level representation of Quantum Numbers and functions like `repeat` and the `if_`.

Here the `if_` function is used to make a classical condition on the application of the CNOTS. The condition is made on $r_i$.

With Classiq, we cannot define $r$ as bit string of 0 and 1, so we define it as integer and extract $r_i$ from it through the arithmetic expression $\lfloor r/2^i \rfloor\, \textrm{mod} \, 2$.

In [7]:
from classiq import *
from classiq.qmod.symbolic import floor
@qfunc
def bitwise_inner_product_mod2(r: CInt, x: QArray, b: QBit):
    repeat(
        x.len,
        lambda i:
            if_(
                floor(r / 2**i) % 2 == 1,
                        # it means r_i == 1, where r_0 is the most sigificant digit,
                        # and r_{x.len-1} is the least signifcant digit
                lambda: CX(x[i], b)
                # I am not sure that it is needed to insert here also "then_= IDENTTITY(b)"
                # in some examples it is not present, in others it's present
            )
    )
# Note: the function if_ has arguments: "condtion", "then", "else_" (optional)
# "then" and "else_" are callable function (lamdda func. or previous defined func.)

#### Implemenation of the step 8b using Classiq Python SDK

The step 8b is the measurement of $x$ register in Hadamard basis, so in Classiq it's just application the **Hadamard transform**.

In Classiq the measrument gates are not present. The measurement is made automatically when the quantum program is executed with on "normal" simulator (not the statevector simulator) or on the real quantum hardware. 

In Classiq the is the builtin quantum function for the Hadamard transform `hadamard_transform`, that is simply the application of an Hadamard gate to each qubit of the qubit array passed as input to the function. 

#### Testing the function of step 7b

The protocol requires that in the step 8b we measure the $x$ register of the state obtained from the step 7b in the Hadamard basis, but before that we neeed to test the step 7b, so will measure the ancilla qubit $b$ to check if the bitwise inner product modulo 2 was actually computed.

In order to test the function `bitwise_inner_product_mod2` we prepare the $x$ register in a state $|x\rangle$.

For test purpose we randomly chose $x$ as unsigned integer. The number has to be the certain size in terms of bits, so we choose a bit size $m$ and choose $x$ in the range $[0,2^{m-1}]$.

The bitstring $r$ has to be the same size bitstring corresponding to $x$, we also randomly choose $r$ as unsigned integer number in the range $[0,2^{m-1}]$.

In the below cell, we define also a classical function to compute the the inner product between $r$ and $x$ to check the correctness of the result.

In [12]:
import math

m=4                    # it is just an example
cx = randrange(0,2**m) # classical value of x; 2**m is excluded
print("cx:",cx)

r = randrange(0,2**m)
print("r:",r)

# classical function of bitwise inner product modulo 2
def classical_bitwise_inner_product_mod2(m,x,r):
    inner_prod=0
    for i in range(m):
        r_i = math.floor(r / 2**i) % 2
        x_i = math.floor(x / 2**i) % 2
        inner_prod += r_i * x_i
    inner_prod = inner_prod % 2
    return inner_prod

@qfunc
def main(b: Output[QBit]): # we set as output argument only the ancilla qbit b, since we want measure only this for testing

    allocate(1,b)
    x = QNum('x')
    allocate_num(m,False,0,x) # if I use prepare_it (instead of inplace_prepare_int) allocate_num is not needed
    inplace_prepare_int(cx,x)
    
    bitwise_inner_product_mod2(r, x, b)

qmod = create_model(main)

quantum_program = synthesize(qmod)

cx: 4
r: 13


In [13]:
#show(quantum_program)

In [14]:
job = execute(quantum_program)
print(
    f"The job on the provider {job.provider} on the backend {job.backend_name} with {job.num_shots} shots is {job.status} can be accessed in the IDE with this URL: {job.ide_url}"
)

The job on the provider Classiq on the backend simulator with 2048 shots is QUEUED can be accessed in the IDE with this URL: https://platform.classiq.io/jobs/7f4d7ca3-170e-4099-a9ab-43fb17adbd23


In [18]:
print("m =",m,", cx =",cx,", r =",r)

results = job.result()[0].value
print("Quantum result:",results.parsed_counts)

prod = classical_bitwise_inner_product_mod2(m,cx,r)
print("Classical result:",prod)


m = 4 , cx = 4 , r = 13
Quantum result: [{'b': 1.0}: 2048]
Classical result: 1
